# ChIP AML PiPeline v2

In [26]:
import os
import pandas as pd
import sys
sys.path.insert(0, '../..')
import itertools
from scipy import stats
import numpy as np

from JKBio.epigenetics import chipseq as chip
from JKBio.utils import helper
import igv
import SimpSOM as sps
from scipy import stats

import seaborn as sns
from matplotlib import cm
from matplotlib import pyplot as plt
from IPython.display import IFrame
import seaborn as sns
from bokeh.plotting import *
import igv

import numba
from numba import jit

from scipy.cluster.hierarchy import linkage, leaves_list
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans, OPTICS
from sklearn.mixture import GaussianMixture
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from IPython.display import IFrame

from pybedtools import BedTool
import pyBigWig

output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
project="Cobinding_ChIP"
version="v3"
merging_version="remove_single"
window="150"

In [11]:
%store -r merged
%store -r chrombed
%store -r mergedpeak
%store -r cols
%store -r annot
%store -r version
%store -r merging_version
%store -r window

## Doing Motif analysis

In [8]:
! gatk FastaAlternateReferenceMaker -R ../../data/ref/Homo_sapiens_assembly38.fasta -O - -L ../../data/$project/ATAC_MV411.mRp.clN_peaks.broadPeak -V input.vcf | fold -w 500 > ../../data/$project/MEME_$version/ATAC.fasta

/bin/sh: 1: gatk: not found


In [27]:
## computing motif across the open region of the genome of MV411 from ATACseq with MEME.mast
#! mkdir ../../data/$project/MEME_$version/ATAC/
! gatk FastaAlternateReferenceMaker -R ../../data/ref/Homo_sapiens_assembly38.fasta -O - -L ../../data/$project/ATAC_MV411.mRp.clN_peaks.broadPeak -V input.vcf | fold -w 500 > ../../data/$project/MEME_$version/ATAC.fasta
# cmd = "export PATH=$HOME/meme/bin:$HOME/meme/libexec/meme-5.1.1:$PATH && mast --oc ../../data/"+project+"/MEME_"+version+"/ATAC/ ../../motif_databases/HUMAN/HOCOMOCOv11_full_HUMAN_mono_meme_format.meme ../../data/"+project+"/MEME_"+version+"/ATAC.fasta"
#! $cmd
#cmd = "export PATH=$HOME/meme/bin:$HOME/meme/libexec/meme-5.1.1:$PATH && fimo --oc ../../data/"+project+"/MEME_"+version+"/ATAC/ ../../motif_databases/HUMAN/HOCOMOCOv11_full_HUMAN_mono_meme_format.meme ../../data/"+project+"/MEME_"+version+"/ATAC.fasta"
#! $cmd

! gff2bed < ../../data/$project/MEME_$version/ATAC/fimo.gff > ../../data/$project/MEME_$version/ATAC/fimo.gff.bed

mkdir: cannot create directory ‘../../data/Cobinding_ChIP/MEME_v3/ATAC/’: File exists
Using motif +AHR_HUMAN.H11MO.0.B of width 9.
Using motif -AHR_HUMAN.H11MO.0.B of width 9.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.9225
Using motif +AIRE_HUMAN.H11MO.0.C of width 18.
Using motif -AIRE_HUMAN.H11MO.0.C of width 18.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=1
Using motif +ALX1_HUMAN.H11MO.0.B of width 12.
Using motif -ALX1_HUMAN.H11MO.0.B of width 12.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=1
Using motif +ALX3_HUMAN.H11MO.0.D of width 11.
Using motif -ALX3_HUMAN.H11MO.0.D of width 11.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=1
Using motif +ALX4_HUMAN.H11MO.0.D of widt

Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.999598
Using motif +BATF_HUMAN.H11MO.1.A of width 11.
Using motif -BATF_HUMAN.H11MO.1.A of width 11.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.996308
Using motif +BC11A_HUMAN.H11MO.0.A of width 17.
Using motif -BC11A_HUMAN.H11MO.0.A of width 17.
Motif matches with p-value >= 3.5e-05 have been dropped to reclaim memory.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.963978
Using motif +BCL6B_HUMAN.H11MO.0.D of width 11.
Using motif -BCL6B_HUMAN.H11MO.0.D of width 11.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=1
Using motif +BCL6_HUMAN.H11MO.0.A of width 13.
Using motif -BCL6_HUMAN.H11MO.0.A of width 13.
Computing q-values.
  Estima

Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.999596
Using motif +CRX_HUMAN.H11MO.0.B of width 13.
Using motif -CRX_HUMAN.H11MO.0.B of width 13.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=1
Using motif +CUX2_HUMAN.H11MO.0.D of width 11.
Using motif -CUX2_HUMAN.H11MO.0.D of width 11.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=1
Using motif +CXXC1_HUMAN.H11MO.0.D of width 7.
Using motif -CXXC1_HUMAN.H11MO.0.D of width 7.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.992609
Using motif +DBP_HUMAN.H11MO.0.B of width 11.
Using motif -DBP_HUMAN.H11MO.0.B of width 11.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated p

Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.909624
Using motif +EHF_HUMAN.H11MO.0.B of width 15.
Using motif -EHF_HUMAN.H11MO.0.B of width 15.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.953957
Using motif +ELF1_HUMAN.H11MO.0.A of width 14.
Using motif -ELF1_HUMAN.H11MO.0.A of width 14.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.982222
Using motif +ELF2_HUMAN.H11MO.0.C of width 13.
Using motif -ELF2_HUMAN.H11MO.0.C of width 13.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.946423
Using motif +ELF3_HUMAN.H11MO.0.A of width 14.
Using motif -ELF3_HUMAN.H11MO.0.A of width 14.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_

Motif matches with p-value >= 3.5e-05 have been dropped to reclaim memory.
Motif matches with p-value >= 1.2e-05 have been dropped to reclaim memory.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.911391
Using motif +FLI1_HUMAN.H11MO.1.A of width 13.
Using motif -FLI1_HUMAN.H11MO.1.A of width 13.
Motif matches with p-value >= 3.3e-05 have been dropped to reclaim memory.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.960468
Using motif +FOSB_HUMAN.H11MO.0.A of width 9.
Using motif -FOSB_HUMAN.H11MO.0.A of width 9.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.999196
Using motif +FOSL1_HUMAN.H11MO.0.A of width 12.
Using motif -FOSL1_HUMAN.H11MO.0.A of width 12.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating p

...

Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.949548
Using motif +TFE3_HUMAN.H11MO.0.B of width 10.
Using motif -TFE3_HUMAN.H11MO.0.B of width 10.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.954495
Using motif +TFEB_HUMAN.H11MO.0.C of width 9.
Using motif -TFEB_HUMAN.H11MO.0.C of width 9.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=1
Using motif +TGIF1_HUMAN.H11MO.0.A of width 7.
Using motif -TGIF1_HUMAN.H11MO.0.A of width 7.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.954625
Using motif +TGIF2_HUMAN.H11MO.0.D of width 11.
Using motif -TGIF2_HUMAN.H11MO.0.D of width 11.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  

Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.924552
Using motif +Z354A_HUMAN.H11MO.0.C of width 24.
Using motif -Z354A_HUMAN.H11MO.0.C of width 24.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.991122
Using motif +ZBED1_HUMAN.H11MO.0.D of width 13.
Using motif -ZBED1_HUMAN.H11MO.0.D of width 13.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.979455
Using motif +ZBT14_HUMAN.H11MO.0.C of width 9.
Using motif -ZBT14_HUMAN.H11MO.0.C of width 9.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.97575
Using motif +ZBT17_HUMAN.H11MO.0.A of width 19.
Using motif -ZBT17_HUMAN.H11MO.0.A of width 19.
Motif matches with p-value >= 3e-05 have been dropped to reclaim memory.
Motif matches with p-v

  Estimated pi_0=0.999899
Using motif +ZN140_HUMAN.H11MO.0.C of width 24.
Using motif -ZN140_HUMAN.H11MO.0.C of width 24.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.986196
Using motif +ZN143_HUMAN.H11MO.0.A of width 22.
Using motif -ZN143_HUMAN.H11MO.0.A of width 22.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.967758
Using motif +ZN148_HUMAN.H11MO.0.D of width 15.
Using motif -ZN148_HUMAN.H11MO.0.D of width 15.
Motif matches with p-value >= 3.2e-05 have been dropped to reclaim memory.
Motif matches with p-value >= 9.2e-06 have been dropped to reclaim memory.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.922105
Using motif +ZN214_HUMAN.H11MO.0.C of width 22.
Using motif -ZN214_HUMAN.H11MO.0.C of width 22.
Computing q-values.
  Estimating pi_0 from a un

Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.957952
Using motif +ZN410_HUMAN.H11MO.0.D of width 13.
Using motif -ZN410_HUMAN.H11MO.0.D of width 13.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=1
Using motif +ZN418_HUMAN.H11MO.0.C of width 17.
Using motif -ZN418_HUMAN.H11MO.0.C of width 17.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.970879
Using motif +ZN418_HUMAN.H11MO.1.D of width 12.
Using motif -ZN418_HUMAN.H11MO.1.D of width 12.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.980541
Using motif +ZN423_HUMAN.H11MO.0.D of width 15.
Using motif -ZN423_HUMAN.H11MO.0.D of width 15.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating 

Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.941183
Using motif +ZNF18_HUMAN.H11MO.0.C of width 12.
Using motif -ZNF18_HUMAN.H11MO.0.C of width 12.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.962824
Using motif +ZNF41_HUMAN.H11MO.0.C of width 24.
Using motif -ZNF41_HUMAN.H11MO.0.C of width 24.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.961582
Using motif +ZNF41_HUMAN.H11MO.1.C of width 12.
Using motif -ZNF41_HUMAN.H11MO.1.C of width 12.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Estimating pi_0.
  Estimated pi_0=0.979027
Using motif +ZNF76_HUMAN.H11MO.0.C of width 22.
Using motif -ZNF76_HUMAN.H11MO.0.C of width 22.
Computing q-values.
  Estimating pi_0 from a uniformly sampled set of 10000 p-values.
  Esti

In [9]:
## What are the motifs of our CRC members in ATACseq but not in our matrix
merged_motif = pd.read_csv('../../data/MEME/merged/fimo.gff.bed', sep='\t',skiprows=0,index_col=None, names=['pos', 'relstart', 'relend', "name", "strength","strand",'fimo','nucleotide_motif',".","data"])

merged_motif['tf'] = [i.split('_HUMAN')[0] for i in merged_motif.name]
merged_motif.relstart = merged_motif.relstart.astype(int)
merged_motif.start = merged_motif.start.astype(int)
merged_motif.relend = merged_motif.relend.astype(int)
merged_motif.start = merged_motif.start+merged_motif.relstart
merged_motif.end = merged_motif.start+merged_motif.relend
merged_motif['chrom'] = [i.split(':')[0][3:] for i in merged_motif.pos]
merged_motif['start'] = [i.split(':')[1].split('-')[0] for i in merged_motif.pos]

In [35]:
rn={'HOXA9':'HXA9',
 'SREBF1':'SRBP1',
 'STAT5B':'STA5B',
 'ZEB2':"ZEB1",
 'ZNF281':'ZN281'}
proteins = []
for i in merged.columns[cols:]:
    for k,v in rn.items():
        i = i.replace(k,v)
    proteins.append(i)
set(proteins) - set(merged_motif.tf)

{'AFF4',
 'ATAC',
 'BRD4',
 'CDK13',
 'CDK9',
 'FLAG_GFI1',
 'FLAG_IRF2BP2',
 'FLAG_MEF2C',
 'FLAG_MEF2D',
 'FLAG_SPI1',
 'FOXP1_mot',
 'H3K27ac',
 'H3K27me3',
 'H3K36me2',
 'H3K36me3',
 'H3K4me1',
 'H3K4me3',
 'H3K79me2',
 'H3K9ac',
 'HEX',
 'LDB1',
 'LMO2',
 'MED1',
 'MLL_KTM2A',
 'MYBL2',
 'PLAGL2',
 'POLII',
 'PSER2',
 'SMC1',
 'WDR5',
 'ZMYND8',
 'activation',
 'promoters',
 'regular_enhancer',
 'repression',
 'super_enhancer'}

In [44]:
merged_motif = merged_motif[merged_motif.tf.isin(proteins)]
merged_motif['pval'] = [float(i.split('pvalue=')[1].split(';')[0]) for i in merged_motif.data]
merged_motif['seq'] = [i.split('sequence=')[1].split(';')[0] for i in merged_motif.data]
merged_motif = merged_motif[['chrom', 'start', 'end', "strength", 'pval','strand', 'tf','seq']].sort_values(by=['chrom','start','strength']).reset_index(drop = True)

In [49]:
merged_motif.to_csv('../results/'+project+'/'+version+'_merged_motif.bed', sep='\t')

In [28]:
merged_motif

,chrom,start,end,strength,pval,strand,tf,seq
0,1,10282,10707,40.5,8.970000e-05,-,RUNX2,GGGTTGGGGTTAGG
1,1,10288,10719,41.6,6.910000e-05,-,RUNX2,GGGTTGGGGTTGGG
2,1,10294,10731,41.6,6.910000e-05,-,RUNX2,GGGTTGGGGTTGGG
3,1,10300,10743,41.6,6.910000e-05,-,RUNX2,GGGTTGGGGTTGGG
4,1,10306,10755,41.6,6.910000e-05,-,RUNX2,GGGTTGGGGTTGGG
...,...,...,...,...,...,...,...,...
2057056,Y,56876562,56876630,67.0,1.970000e-07,-,IRF8,GATAGCGGAAGTGAAAATCA
2057057,Y,56876568,56876635,52.9,5.060000e-06,-,ELF2,ATAGCGGAAGTGA
2057058,Y,56876668,56876833,46.0,2.490000e-05,-,MYC,GCCATGTGTTT
2057059,Y,56878627,56878678,45.3,2.980000e-05,+,GATA2,AAAGATAAGAG


In [18]:
merged_motif = merged_motif.rename(columns={'chr':'chrom'})

In [37]:
a = {}
for k,v in rn.items():
    a[v] = k 
rn = a
merged_motif['tf'] = merged_motif['tf'].replace(rn)

In [23]:
merged_motif.chrom = merged_motif.chrom.astype(str)

In [32]:
val

'ZN281'

In [33]:
merged.columns

Index(['chrom', 'start', 'end', 'peak_number', 'foldchange', '-log10pvalue',
       '-log10qvalue', 'relative_summit_pos', 'CEBPA', 'HOXA9', 'PLAGL2',
       'MEIS1', 'ETV6', 'HEX', 'FLAG_GFI1', 'RUNX1', 'FOSL2', 'FLI1', 'GATA2',
       'SP1', 'E2F3', 'ZNF281', 'SREBF1', 'LMO2', 'STAT5B', 'ZMYND8',
       'FLAG_MEF2C', 'LYL1', 'MYC', 'TFAP4', 'RUNX2', 'MAX', 'IRF8', 'MYB',
       'ZEB2', 'RXRA', 'FLAG_MEF2D', 'activation', 'repression', 'ELF2',
       'JUND', 'FLAG_SPI1', 'FLAG_IRF2BP2', 'SMC1', 'LDB1', 'H3K79me2', 'WDR5',
       'H3K27ac', 'H3K27me3', 'MYBL2', 'CDK9', 'H3K4me3', 'FOXP1', 'MLL_KTM2A',
       'RARA', 'IKZF1', 'AFF4', 'CTCF', 'BRD4', 'CEBPB', 'H3K36me2', 'PSER2',
       'H3K36me3', 'H3K9ac', 'CDK13', 'H3K4me1', 'POLII', 'MED1', 'ATAC',
       'promoters', 'super_enhancer', 'regular_enhancer', 'FOXP1_mot'],
      dtype='object')

In [40]:
for val in set(merged_motif.tf):
    merged[val+'_mot'] = chip.putInBed(merged[merged.columns[:cols]], merged_motif[merged_motif.tf==val], window=2000)
merged = merged.replace(np.nan,0)

0 were merged into conscensus


KeyboardInterrupt: 

In [4]:
merged_motif = pd.read_csv('../results/'+project+'/'+version+'_merged_motif.bed', sep='\t')

/home/jeremie/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
## what enrichment do we have in each group? what enrichment do we have for each ChipSeq?

In [ ]:
## computing predicted motif for each TF from Chip data MEME-Chip

In [ ]:
## comparing with the litterature using MEME.tomtom

In [ ]:
### loading motifs from 

## Assigning genes

### based on closest expressed gene

In [ ]:
chiphelper.AssignToClosestExpressed()

In [ ]:
### recompute cobinding based on this.

In [ ]:
## redo the plots. do we get better plots?/correlations?...

### based on the ABC model

![](images/ABCtitle.png)

They tested a new model based on and validated by CRISPRi-FlowFISH which is basically able to find enhancer mapping to genes. 
They used it to compute their model's Accuracy and found a 70% accuracy compared to less than 50% for closest expressed gene. 

Way to integrate our HiC data (need ATAC-seq like data as well, but openly available) 


![](images/ABCmodel.png)

In [ ]:
Helper.scatter(TSNE(2,5).fit_transform(data.T), labels=zones.columns[11:],colors=labels)

In [ ]:
### recompute cobinding based on this.

In [ ]:
### redo the plots. do we get better looking plots?

In [ ]:
### compare presence of CTCF and transcription of linked RNA

## Predict

In [ ]:
## enrichment at the gene set level

In [ ]:
## are our cobinding clusters enriched in some sets of genes / CRC? 

In [ ]:
## make a linear model to with marks and cobinding data + motifs and TFs+cobinding motifs + expression of cobound proteins + expression , can we predict expression/dependenccy?

In [ ]:
## can we predict better for some set of genes / CRCs?

In [ ]:
## do the same on closest expressed gene

In [ ]:
## do this prediction on each enhancer.what is the best predicting enhancer? do that correlate with ABC model data?

In [ ]:
## do we get, for some gene, better single enhancer prediction?

In [ ]:
## what are the gene sets enriched in each clusters?? (based on TF cobinding or based on TF cobinding + )

## RNP data

In [ ]:
## can we predict RNP data (setting this TF to zero and lookingg at expected RNA change)
## do we see genes with opposite effects? 

In [ ]:
## if we add RNP data, can we increase our model's prediction? (we have expression change and we set all RNPed-TF values to 0) 

In [ ]:
### to predict remaining X% RNA expression
### to predict remaining X TFs RNP
### to predict regular RNA expression

In [ ]:
## same thing with filtering base on motif presence (actual DNA binding)
## same thing with closest expressed gene

In [ ]:
## Do on all cell lines

In [ ]:
## repeat the process across all samples with H3K27ac+RNAseq data we have.

In [ ]:
## call mutations from H3k27ac data

In [ ]:
## MEME analysis of likely bound TFs, except if TF is not expressed

In [ ]:
## compute enrichments

In [ ]:
## from same gene assignements, as found on the general consensus peak set, can we find good dependency/expression prediction

### Compare data with other labs (H3K27, HiC..)

we need to redo everything for similar normal cell type, getting TFs based on the CRC (find it with CRCmapper or on litterature)